# Day 7 : <a href="https://adventofcode.com/2023/day/7">Camel cards</a>

## Part 1

In this problem, we're asked to carry out ordering on a set of cards. The possible cards are (in descending order):

`A`, `K`, `Q`, `J`, `T`, `9`, `8`, `7`, `6`, `5`, `4`, `3`, `2`, `1`

Each set of cards, also called a <i>hand</i> has a corresponding bid. We're asked to find the total winnings of a set of <i>hands</i>. Where each bid corresponding to a hand is multiplied by it's ranking. Where the rankings follow regular poker rules.

As test input data we are given

In [6]:
testData = """32T3K 765
T55J5 684
KK677 28
KTJJT 220
QQQJA 483"""

Here the hand `32T3K` has a bid of `765`. If this were to be the worst card of our hand, it would have a value of $1\times 765$. If it were the best of the 5 cards, it would have a value of $5\times 765=3825$. We note the total winnings by the sum of all individual hand winnings. And for our `testData` we have a `testResult` of

In [7]:
testResult = 6440

We begin by importing regular modules

In [8]:
import numpy as np
import backend
import re

There are different kinds of hands. First, we write a function that classifies these hands. We do so by assigning numerical values to each type of hand, which we can sort by later.

In [9]:
high_card = 1
one_pair = 2
two_pair = 3
three_of_a_kind = 4
full_house = 5
four_of_a_kind = 6
five_of_a_kind = 7

Let us first define a function that returns a dictionary with all cards and their relative number of appearances.

In [24]:
def cardsInHand(hand : str) -> dict:
    cards = np.array([card for card in hand], dtype = str) # Create a numpy array
    cardsDict = dict() # Create an empty dictionary first
    for card in cards : cardsDict.update({card : np.sum(cards == card)})
    return cardsDict

cardsInHand("AA36J")

{'A': 2, '3': 1, '6': 1, 'J': 1}